# Imports

In [1]:
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib as mpl
from sqlalchemy import create_engine
import psycopg2

%matplotlib inline  

In [ ]:
# Load credentials from json file
with open("credentials.json", "r") as file:  
    creds = json.load(file)

USERNAME = creds['USERNAME']
PASSWORD = creds['PASSWORD']

# Connect to the db

In [2]:
#create the engine and connection
engine = create_engine('postgresql+psycopg2:'\
                       USERNAME\
                       PASSWORD\
                       '@49.12.227.17:5432/'\
                       'lahman2017')

print('Database session created')
con = engine.connect()

Database session created


In [3]:
df = pd.read_sql_query("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
        """, con)
df

,table_name
0,allstarfull
1,appearances
2,awardsmanagers
3,awardsplayers
4,awardssharemanagers
5,awardsshareplayers
6,batting
7,battingpost
8,collegeplaying
9,fielding


# Run queries

## Q1 Which player hit the most home runs in 2002?

In [4]:
# USE SET search_path = lahman2017 so you don't have to reference the schema all the time!

df = pd.read_sql_query("""
    SELECT
        namefirst,
        namelast
    From master
    WHERE playerid = 
    (SELECT playerid
    FROM batting
    WHERE yearid = 2002
    ORDER BY HR DESC
    LIMIT 1);
        """, con)

df

,namefirst,namelast
0,Alex,Rodriguez


In [5]:
# with join
df = pd.read_sql_query("""
    SELECT
        mas.namefirst,
        mas.namelast,
        bat.hr AS HomeRuns
    FROM batting bat
    LEFT JOIN master mas
    ON mas.playerid = bat.playerid
    WHERE yearid=2002
    ORDER BY hr DESC
    LIMIT 1;
        """, con)

df
#--> ANSWER: Alex Rodriguez with 57 homeruns in 2002.

,namefirst,namelast,homeruns
0,Alex,Rodriguez,57


## Q2 Which team spent the most/least money on player salaries in 2002?

In [6]:
## most money
df = pd.read_sql_query("""
    SELECT
        tms.name AS Team, 
        sum(sal.salary) AS TotalSalaries
    FROM salaries AS sal
    INNER JOIN teams tms
    ON tms.teamid = sal.teamid AND tms.yearid = sal.yearid
    WHERE sal.yearid = 2002
    GROUP BY sal.teamid, Team
    ORDER BY sum(sal.salary)  DESC
    LIMIT 1;
""", con)

df
# --> ANSWER: New York Yankees with USD 125,928,583.

,team,totalsalaries
0,New York Yankees,125928583.0


In [7]:
## least money
df = pd.read_sql_query("""
    SELECT
        tms.name AS Team, 
        sum(sal.salary) AS TotalSalaries
    FROM salaries AS sal
    INNER JOIN teams tms
    ON tms.teamid = sal.teamid AND tms.yearid = sal.yearid
    WHERE sal.yearid = 2002
    GROUP BY sal.teamid, Team
    ORDER BY sum(sal.salary)  ASC
    LIMIT 1;
""", con)

df
# --> ANSWER: Tampa Bay Devil Rays with USD 34,380,000.

,team,totalsalaries
0,Tampa Bay Devil Rays,34380000.0


## Q3 Which player averaged the fewest at bats between home runs in 2002?

In [8]:
df = pd.read_sql_query("""
    SELECT
        mas.namefirst,
        mas.namelast,
        bat.hr AS HomeRuns,
        bat.ab AS AtBats,
        CAST(bat.ab AS FLOAT)/bat.hr AS Avg_AB_btw_HR
    FROM batting bat
    LEFT JOIN master mas
    ON mas.playerid = bat.playerid
    WHERE 
        yearid=2002 AND
        bat.hr > 1 --between implies at least two HR
    ORDER BY Avg_AB_btw_HR ASC
    LIMIT 1;
    """, con)

df
# --> ANSWER: Barry Bonds with an average of 8.76 at bats between homeruns in 2002.

,namefirst,namelast,homeruns,atbats,avg_ab_btw_hr
0,Barry,Bonds,46,403,8.76087


## Q4 Which player in 2002 had the highest on-base percentage?

In [9]:
df = pd.read_sql_query("""
    SELECT
        mas.namefirst,
        mas.namelast,
        bat.ab AS AtBats,
        bat.h as Hits,
        bat.bb as BasesOnBalls,
        bat.hbp AS HitByPitch,
        bat.sf AS SacrificeFlies,
        CAST(bat. h +bat.bb +bat.hbp AS FLOAT)/CAST(bat.ab+bat.bb+bat.hbp+bat.sf AS FLOAT) * 100 AS OBP_in_percent
    FROM batting bat
    LEFT JOIN master mas
    ON mas.playerid = bat.playerid
    WHERE 
        yearid=2002 AND
        bat.hr > 0 -- otherwise OBP is not meaningful
    ORDER BY OBP_in_percent DESC
    LIMIT 5;
    """, con)

df
# --> ANSWER: The player with the highest on-base percentage (and at least one homerun) 
#             in 2002 is Barry Bonds with an OBP of 58.17%.

,namefirst,namelast,atbats,hits,basesonballs,hitbypitch,sacrificeflies,obp_in_percent
0,Barry,Bonds,403,149,198,9.0,2.0,58.169935
1,Mike,Kinkade,50,19,4,6.0,0.0,48.333333
2,Mark,Loretta,66,28,9,0.0,2.0,48.051948
3,Brian,Giles,497,148,135,7.0,5.0,45.031056
4,Manny,Ramirez,436,152,73,8.0,1.0,44.980695


## Q5 Which Yankees pitcher had the most wins in a season in the 00’s?

In [10]:
df = pd.read_sql_query("""

    SELECT
        mas.namefirst,
        mas.namelast,
        tms.name AS Team,
        pit.yearid AS YearID,
        pit.w AS Wins
    FROM pitching pit
    JOIN teams tms
    ON pit.teamid = tms.teamid AND pit.yearid = tms.yearid
    JOIN master mas
    ON mas.playerid = pit.playerid
    WHERE
        tms.teamid = 'NYA' AND
        pit.yearid > 1999 AND
        pit.yearid <2010
    ORDER BY pit.w DESC
    LIMIT 2;
    """, con)

df
# --> ANSWER: This is Andy Pettitte who achieved 21 wins as pitcher for the Yankees in the year 2003.

,namefirst,namelast,team,yearid,wins
0,Andy,Pettitte,New York Yankees,2003,21
1,Roger,Clemens,New York Yankees,2001,20


## Which Yankees pitcher had the most wins between 1999 and 2009?

In [11]:
df = pd.read_sql_query("""

    SELECT t1.namefirst, t1.namelast, sum(wins)
    FROM
    (SELECT
        mas.namefirst,
        mas.namelast,
        tms.name AS Team,
        pit.yearid AS YearID,
        pit.w AS Wins
    FROM pitching pit
    JOIN teams tms
    ON pit.teamid = tms.teamid AND pit.yearid = tms.yearid
    JOIN master mas
    ON mas.playerid = pit.playerid
    WHERE
        tms.teamid = 'NYA' AND
        pit.yearid > 1999 AND
        pit.yearid <2010) as t1
    GROUP BY t1.namefirst, t1.namelast
    ORDER BY sum(wins) DESC
    LIMIT 5;
     """, con)

df
# --> ANSWER: Mike Mussina had 123 wins between 1999 and 2009

,namefirst,namelast,sum
0,Mike,Mussina,123.0
1,Andy,Pettitte,111.0
2,Roger,Clemens,69.0
3,Chien-Ming,Wang,55.0
4,Mariano,Rivera,45.0


## Q6 In the 2000’s, did the Yankees draw more or fewer walks (Base-on-Balls or BB) as the decade went on?
- with alternative connection

In [12]:
# we create a connection cursor to retrieve the date with the execute function (alternative way to read_sql_query)
with engine.connect() as con:
    rs = con.execute("""
    
    SELECT
        teamid,
        yearid,
        sum(BB)
    FROM batting 
    INNER JOIN master
    ON master.playerid = Batting.playerid
    WHERE
        yearid >= 2000
        AND
        yearid < 2010
        AND
        teamID = 'NYA'
    GROUP BY yearid, teamID;
    """)
    yankees = rs.fetchall()
    
yankees

[('NYA', 2000, Decimal('631')),
 ('NYA', 2001, Decimal('519')),
 ('NYA', 2002, Decimal('640')),
 ('NYA', 2003, Decimal('684')),
 ('NYA', 2004, Decimal('670')),
 ('NYA', 2005, Decimal('637')),
 ('NYA', 2006, Decimal('649')),
 ('NYA', 2007, Decimal('637')),
 ('NYA', 2008, Decimal('535')),
 ('NYA', 2009, Decimal('663'))]

In [13]:
df = pd.DataFrame(yankees, columns = ['Team','Year', 'Walks'])
df

,Team,Year,Walks
0,NYA,2000,631
1,NYA,2001,519
2,NYA,2002,640
3,NYA,2003,684
4,NYA,2004,670
5,NYA,2005,637
6,NYA,2006,649
7,NYA,2007,637
8,NYA,2008,535
9,NYA,2009,663
